In [25]:
import pandas as pd
import numpy as np
import faiss
import lightgbm as lgb
import pickle
import os

In [26]:


BASE_DIR = r"D:\Machine_learning_projects\Product_recommendation_system"
ARTIFACTS_DIR = os.path.join(BASE_DIR, "artifacts")


In [27]:

index = faiss.read_index(os.path.join(ARTIFACTS_DIR, "faiss.index"))


rank = lgb.Booster(
    model_file=os.path.join(ARTIFACTS_DIR, "lgbm_ranker.txt")
)

with open(os.path.join(ARTIFACTS_DIR, "recommendation.pkl"), "rb") as f:
    meta = pickle.load(f)

train_df = meta["train_df"]
train_embeddings = meta["train_embedding"]
FEATURES = meta["features"]


In [28]:
train_df.head(3)

,uniq_id,product_id,product_category,product_brand,product_name,product_price,product_description,product_currency,product_tags,product_reviews_count,...,category_mean_rating,crawl_month,crawl_dayofweek,categorical_L1,categorical_L2,categorical_L3,categorical_L4,text,bayesain_rating,popularity
0,2612,971,premium beauty premium makeup premium lips pre...,NARS,nars velvet matte lip pencil endangered red,11.25,,USD,nars velvet matte lip pencil endangered red wa...,22.0,...,4.29,10,4,Premium Beauty,Premium Makeup,Premium Lips,Premium Lip Liner,nars velvet matte lip pencil endangered red NA...,4.310055,3.135494
1,3524,2947,premium beauty premium makeup premium nail pol...,OPI,opi nail polish lacquer holiday hello kitty 5o...,10.95,,USD,opi nail polish lacquer holiday hello kitty 5o...,4.0,...,3.00,7,4,Premium Beauty,Premium Makeup,Premium Nail Polish & Care,Premium Nail Kits,opi nail polish lacquer holiday hello kitty 5o...,4.268774,1.609438
2,219,4388,beauty beauty by top brands l oreal l oreal ma...,L'Oreal Paris,loreal colour riche lipstick 183 pink vinyl,6.41,183 pink vinylLp Generic Loreal Paris Colou.,USD,l oreal colour riche lipstick 183 pink vinyl w...,38.0,...,4.02,7,4,Beauty,Beauty by Top Brands,L'Oreal,L'Oreal Makeup,loreal colour riche lipstick 183 pink vinyl L'...,4.259866,3.663562


In [33]:
index.ntotal

3841

In [34]:
train_embeddings.shape

(3841, 384)

In [35]:
train_df.columns

Index(['uniq_id', 'product_id', 'product_category', 'product_brand',
       'product_name', 'product_price', 'product_description',
       'product_currency', 'product_tags', 'product_reviews_count',
       'crawl_date', 'crawl_time', 'rating_for_model',
       'product_image_url_jpeg', 'popularity_score', 'brand_mean_rating',
       'category_mean_rating', 'crawl_month', 'crawl_dayofweek',
       'categorical_L1', 'categorical_L2', 'categorical_L3', 'categorical_L4',
       'text', 'bayesain_rating', 'popularity'],
      dtype='object')

In [29]:

FEATURES = FEATURES


def get_candidates(query_embedding, k=30):
    scores, indices = index.search(query_embedding.reshape(1, -1), k)
    return indices[0], scores[0]


def recommend_product(product_name, top_n=5):
    if product_name not in train_df["product_name"].values:
        print("Product not found! Try a valid product from training data.")
        return None
    

    id = train_df[train_df["product_name"] == product_name].index[0]
    

    cand_id, sims = get_candidates(train_embeddings[id])

    candidates = []
    features = []
    for c_id, sim in zip(cand_id, sims):
        if c_id == id:
            continue  
        candidates.append(c_id)
        features.append([
            sim,
            train_df.loc[c_id, "bayesain_rating"],
            train_df.loc[c_id, "popularity"],
            train_df.loc[c_id, "rating_for_model"],
            int(train_df.loc[c_id, "product_category"] == train_df.loc[id, "product_category"])
        ])
    
    features = pd.DataFrame(features, columns = FEATURES)
    
 
    scores = rank.predict(features)
    

    results = train_df.loc[candidates].copy()
    results["score"] = scores
    

    return results.sort_values("score", ascending=False).head(top_n)[
        ["product_name", "product_image_url_jpeg", "product_category", "rating_for_model", "product_reviews_count", "score"]
    ]


In [32]:
print(recommend_product("loreal colour riche lipstick 183 pink vinyl" , top_n=5))

                                           product_name  \
3831                 loreal colour riche serum lipstick   
2642  loreal paris colour riche collection exclusive...   
188    eternally mauve 520 by loreal for women lipstick   
2927  loreal paris colour riche monos eyeshadow matt...   
1837  loreal paris colour riche original satin lipst...   

                                 product_image_url_jpeg  \
3831  https://i5.walmartimages.com/asr/3ab6533c-1141...   
2642  https://i5.walmartimages.com/asr/c654a699-1a19...   
188   https://i5.walmartimages.com/asr/9e2a84d3-6df8...   
2927  https://i5.walmartimages.com/asr/1cfac04f-2b53...   
1837  https://i5.walmartimages.com/asr/76d82d8d-ff90...   

                                       product_category  rating_for_model  \
3831  beauty beauty by top brands l oreal l oreal ma...              4.02   
2642  beauty beauty by top brands l oreal l oreal ma...              4.02   
188   beauty beauty by top brands l oreal l oreal ma...    